In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [7]:
aud = np.load('data/shw/aud_feats/audio_00000.npz', allow_pickle=True)['features']

In [8]:
aud.mean(0).reshape((-1, 1)).shape

(159744, 1)

In [10]:
vid = np.load('data/shw/vid_feats/video_00000.npz')['features']

In [11]:
vid.shape

(156, 4096)

In [14]:
def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    def _dtype_feature(ndarray):
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print ("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print ("Writing {} done!".format(result_tf_file))

In [21]:
dataset = 'shw'
num_to_read = 1

for i in range(num_to_read):
    batch = np.zeros((1, 16*4096))
    for j in range(1):
        vid = np.load('data/'+dataset+'/vid_feats/video_%05d.npz'%(1*i+j))['features']
        batch[j] = vid[32:48].ravel()
    np_to_tfrecords(batch, None, './data/'+dataset+'/tfr/video_%02d'%i, verbose=True)

for i in range(num_to_read // 1):  
    batch = np.zeros((1, 16384))
    for j in range(1):
        aud = np.load('data/'+dataset+'/aud_feats/audio_%05d.npz'%(1*i+j), allow_pickle=True)['features']
        batch[j] = aud.mean(0)[2*16384:3*16384]
    np_to_tfrecords(batch, None, './data/'+dataset+'/tfr/audio_%02d'%i, verbose=True)

Serializing 1 examples into ./data/shw/tfr/video_00.tfrecords
Writing ./data/shw/tfr/video_00.tfrecords done!
Serializing 1 examples into ./data/shw/tfr/audio_00.tfrecords
Writing ./data/shw/tfr/audio_00.tfrecords done!


In [22]:
vid = np.load('data/shw/vid_feats/video_00000.npz')['features'][16:32]

In [23]:
from scipy.io import wavfile

In [25]:
aud = np.load('data/shw/aud_feats/audio_00000.npz', allow_pickle=True)['features'][0][16384:2*16384]
wavfile.write('test.wav', 16000, aud.astype(np.float32))

In [27]:
test=np.loadtxt('generated_at_epoch3200.csv', delimiter=',')

OSError: generated_at_epoch3200.csv not found.

In [28]:
test[0].shape

NameError: name 'test' is not defined

In [9]:
wavfile.write('test_drum2.wav', 16000, test[0].astype(np.float32))